# 🏗️ EXO_03_PRODUCTION — Batch Production Notebook

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 SCENOGRAPHY DOCK — PRODUCTION BATCH                          ║
║           Construction Automatisée Environnements 3D                         ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Mission:** Construire tous les environnements d'une production vidéo

## 1. Configuration Production

In [ ]:
import os
import sys
import json
from pathlib import Path
from datetime import datetime

# ═══════════════════════════════════════════════════════════════════════════
# CONFIGURATION — MODIFIER SELON VOTRE PRODUCTION
# ═══════════════════════════════════════════════════════════════════════════

DRIVE_ROOT = Path("/content/drive/MyDrive/DRIVE_EXODUS_V2")
# DRIVE_ROOT = Path("/home/EXODUS-V2")  # Local dev

# Fichier de production (généré par CORTEX U00)
PRODUCTION_PLAN = "PRODUCTION_PLAN.JSON"

# Options de traitement
SCENE_IDS = None  # None = toutes les scènes, ou [1, 2, 3] pour filtrer
VERBOSE = True

# ═══════════════════════════════════════════════════════════════════════════

UNIT_ROOT = DRIVE_ROOT / "03_SCENOGRAPHY_DOCK"
CODEBASE = UNIT_ROOT / "CODEBASE"
IN_MAPS = UNIT_ROOT / "IN_MAPS"
OUT_ENVIRONMENTS = UNIT_ROOT / "OUT_ENVIRONMENTS"

sys.path.insert(0, str(CODEBASE))

print("═" * 70)
print("   FRÉGATE 03_SCENOGRAPHY — MODE PRODUCTION")
print(f"   {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("═" * 70)
print(f"\nDrive Root: {DRIVE_ROOT}")
print(f"Production Plan: {PRODUCTION_PLAN}")

## 2. Validation Pré-Production

In [ ]:
# Vérifier tous les prérequis
errors = []
warnings = []

# Check Blender
BLENDER_PATH = DRIVE_ROOT / "EXODUS_AI_MODELS" / "blender-4.0.0-linux-x64" / "blender"
if BLENDER_PATH.exists():
    print("✓ Blender 4.0 disponible")
else:
    errors.append(f"Blender introuvable: {BLENDER_PATH}")

# Check Production Plan
plan_path = IN_MAPS / PRODUCTION_PLAN
if plan_path.exists():
    with open(plan_path, 'r') as f:
        plan = json.load(f)
    scene_count = len(plan.get('scenes', []))
    print(f"✓ Production Plan: {scene_count} scènes")
else:
    errors.append(f"Production Plan introuvable: {plan_path}")

# Check HDRI Library
hdri_path = IN_MAPS / "hdri_library"
hdri_files = list(hdri_path.glob("*.hdr")) + list(hdri_path.glob("*.exr")) if hdri_path.exists() else []
if hdri_files:
    print(f"✓ HDRI Library: {len(hdri_files)} fichiers")
else:
    warnings.append("Aucun HDRi trouvé — fallback éclairage utilisé")

# Check Assets
assets_path = IN_MAPS / "environment_assets"
asset_files = list(assets_path.glob("**/*.blend")) + list(assets_path.glob("**/*.glb")) if assets_path.exists() else []
if asset_files:
    print(f"✓ Environment Assets: {len(asset_files)} fichiers")
else:
    warnings.append("Aucun asset environnement — placeholders utilisés")

# Résumé
print("\n" + "─" * 50)
if errors:
    print("\n❌ ERREURS BLOQUANTES:")
    for e in errors:
        print(f"   • {e}")
    print("\n⛔ Production impossible. Corrigez les erreurs ci-dessus.")
else:
    print("\n✅ Tous les prérequis validés")
    if warnings:
        print("\n⚠️ Avertissements:")
        for w in warnings:
            print(f"   • {w}")

## 3. Analyse du Plan de Production

In [ ]:
# Analyser les besoins
if 'plan' in dir():
    print("═" * 50)
    print("   ANALYSE DU PLAN DE PRODUCTION")
    print("═" * 50)
    
    env_types = {}
    lighting_moods = {}
    all_props = set()
    
    for scene in plan.get('scenes', []):
        env = scene.get('environment', {})
        
        # Types d'environnement
        env_type = env.get('type', 'unknown')
        env_types[env_type] = env_types.get(env_type, 0) + 1
        
        # Moods d'éclairage
        mood = env.get('lighting_mood', 'natural')
        lighting_moods[mood] = lighting_moods.get(mood, 0) + 1
        
        # Props
        for prop in env.get('props', []):
            all_props.add(prop)
    
    print(f"\nTypes d'environnement:")
    for t, count in sorted(env_types.items(), key=lambda x: -x[1]):
        print(f"  • {t}: {count} scènes")
    
    print(f"\nMoods d'éclairage:")
    for m, count in sorted(lighting_moods.items(), key=lambda x: -x[1]):
        print(f"  • {m}: {count} scènes")
    
    print(f"\nProps uniques: {len(all_props)}")
    if all_props:
        print(f"  {', '.join(sorted(all_props))}")
else:
    print("Plan de production non chargé")

## 4. 🚀 LANCEMENT PRODUCTION

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# LANCEMENT DE LA PRODUCTION — EXÉCUTION PRINCIPALE
# ═══════════════════════════════════════════════════════════════════════════

if BLENDER_PATH.exists() and plan_path.exists():
    cmd = [
        f"python {CODEBASE}/EXO_03_SCENOGRAPHY.py",
        f"--drive-root {DRIVE_ROOT}",
        f"--production-plan {plan_path}",
        f"--blender-path {BLENDER_PATH}"
    ]
    
    if SCENE_IDS:
        scene_str = ','.join(map(str, SCENE_IDS))
        cmd.append(f"--scene-ids {scene_str}")
    
    if VERBOSE:
        cmd.append("-v")
    
    full_cmd = " ".join(cmd)
    print(f"Commande: {full_cmd}\n")
    print("═" * 70)
    print("   DÉBUT CONSTRUCTION ENVIRONNEMENTS")
    print("═" * 70 + "\n")
    
    !{full_cmd}
    
    print("\n" + "═" * 70)
    print("   FIN CONSTRUCTION")
    print("═" * 70)
else:
    print("⛔ Impossible de lancer la production (prérequis manquants)")

## 5. Rapport de Production

In [ ]:
# Afficher le rapport de production
report_path = OUT_ENVIRONMENTS / "scenography_report.json"

if report_path.exists():
    with open(report_path, 'r') as f:
        report = json.load(f)
    
    print("═" * 70)
    print("   RAPPORT DE PRODUCTION SCENOGRAPHY")
    print("═" * 70)
    
    status = report.get('status', 'UNKNOWN')
    status_icon = "✅" if status == "SUCCESS" else "❌"
    print(f"\nStatus: {status_icon} {status}")
    print(f"Timestamp: {report.get('timestamp', 'N/A')}")
    print(f"Version: {report.get('version', 'N/A')}")
    
    summary = report.get('summary', {})
    print(f"\n--- Résumé ---")
    print(f"Scènes totales: {summary.get('total_scenes', 0)}")
    print(f"Scènes construites: {summary.get('scenes_built', 0)}")
    print(f"HDRi résolus: {summary.get('hdri_resolved', 0)}")
    print(f"Assets résolus: {summary.get('assets_resolved', 0)}")
    
    print(f"\n--- Scènes ---")
    for scene in report.get('scenes', []):
        s_id = scene.get('scene_id')
        s_type = scene.get('environment_type')
        s_mood = scene.get('lighting_mood')
        s_file = scene.get('output_file', 'N/A')
        print(f"  Scene {s_id}: {s_type} ({s_mood}) → {s_file}")
    
    # HDRi manquants
    hdri_info = report.get('hdri_library', {})
    missing_hdri = hdri_info.get('missing', [])
    if missing_hdri:
        print(f"\n⚠️ HDRi manquants: {missing_hdri}")
else:
    print("Rapport de production non trouvé")

## 6. Fichiers Générés

In [ ]:
# Lister tous les fichiers générés
print("═" * 70)
print("   FICHIERS GÉNÉRÉS")
print("═" * 70)

if OUT_ENVIRONMENTS.exists():
    total_size = 0
    files = sorted(OUT_ENVIRONMENTS.glob("*"))
    
    for f in files:
        if f.is_file():
            size_mb = f.stat().st_size / (1024*1024)
            total_size += size_mb
            icon = "🎬" if f.suffix == ".blend" else "📄"
            print(f"{icon} {f.name:40} {size_mb:8.2f} MB")
    
    print(f"\nTotal: {len(files)} fichiers, {total_size:.2f} MB")
else:
    print("Aucun fichier généré")

## 7. Prochaines Étapes

In [ ]:
print("═" * 70)
print("   PROCHAINES ÉTAPES PIPELINE")
print("═" * 70)
print("""
Les environnements sont prêts pour le compositing:

1. ✓ U00_CORTEX    — Plan de production généré
2. ✓ U01_ANIMATION — Avatars animés (actor_animated.blend)
3. ✓ U02_LOGISTICS — Avatars équipés (actor_equipped.blend)
4. ✓ U03_SCENOGRAPHY — Environnements construits (environment_*.blend)

→ U04_PHOTOGRAPHY — Composition finale + Rendu
   • Importer actor_equipped dans environment
   • Configurer caméras
   • Lancer rendu Cycles/EEVEE
""")

---
*EXODUS SYSTEM — Frégate 03_SCENOGRAPHY v1.0.0*